# Local ColabNAS - Exact Configuration

Using the complete flower dataset with your exact specifications.

In [2]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from local_colabnas import LocalColabNAS, test_tflite_model

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.20.0
GPU available: []


In [3]:
# Download and setup dataset - exact as requested
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = Path(data_dir).with_suffix('')

# Fix dataset structure - point to actual flower_photos folder
actual_data_dir = data_dir / 'flower_photos'
if actual_data_dir.exists():
    data_dir = actual_data_dir

print(f"Dataset directory: {data_dir}")
print(f"Classes found: {[d.name for d in data_dir.iterdir() if d.is_dir()]}")

Dataset directory: C:\Users\Chakkaphan\.keras\datasets\flower_photos_extracted\flower_photos
Classes found: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [4]:
# Exact configuration from your request
input_shape = (50,50,3)

#target: STM32L412KBU3
#273 CoreMark, 40 kiB RAM, 128 kiB Flash
peak_RAM_upper_bound = 40960
Flash_upper_bound = 131072
MACC_upper_bound = 2730000 #CoreMark * 1e4

path_to_training_set = data_dir
val_split = 0.3
cache = True
save_path = './trained_models/'

print(f"Configuration:")
print(f"  Input shape: {input_shape}")
print(f"  RAM limit: {peak_RAM_upper_bound} bytes")
print(f"  Flash limit: {Flash_upper_bound} bytes")
print(f"  MACC limit: {MACC_upper_bound}")

Configuration:
  Input shape: (50, 50, 3)
  RAM limit: 40960 bytes
  Flash limit: 131072 bytes
  MACC limit: 2730000


In [5]:
# GPU check (equivalent to !nvidia-smi)
print("GPU Information:")
if tf.config.list_physical_devices('GPU'):
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"  {gpu}")
else:
    print("  No GPU detected - using CPU")

# Initialize ColabNAS (using LocalColabNAS)
colabNAS = LocalColabNAS(
    max_RAM=peak_RAM_upper_bound, 
    max_Flash=Flash_upper_bound, 
    max_MACC=MACC_upper_bound, 
    path_to_training_set=str(path_to_training_set), 
    val_split=val_split, 
    cache=cache, 
    input_shape=input_shape, 
    save_path=save_path
)

print("\nColabNAS initialized successfully!")

GPU Information:
  No GPU detected - using CPU
Initializing LocalColabNAS with 5 classes
Found 3670 files belonging to 5 classes.
Using 2569 files for training.
Found 3670 files belonging to 5 classes.
Using 1101 files for validation.

ColabNAS initialized successfully!


In [6]:
# Search for optimal architecture
print("Starting search...")
path_to_tflite_model = colabNAS.search()

if path_to_tflite_model:
    print(f"\nSearch completed! Model saved at: {path_to_tflite_model}")
else:
    print("\nNo feasible architecture found.")


Epoch 37: val_accuracy did not improve from 0.54587
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6212 - loss: 0.9625 - val_accuracy: 0.5332 - val_loss: 1.2367
Epoch 38/99
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6247 - loss: 0.9594
Epoch 38: val_accuracy did not improve from 0.54587
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6248 - loss: 0.9591 - val_accuracy: 0.5350 - val_loss: 1.2311
Epoch 39/99
77/81 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6259 - loss: 0.9606
Epoch 39: val_accuracy did not improve from 0.54587
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6262 - loss: 0.9589 - val_accuracy: 0.5286 - val_loss: 1.2265
Epoch 40/99
77/81 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6308 - loss: 0.9573
Epoch 40: val_accuracy did not improve from 0.54587
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6309 - loss: 0.9556 - val_accuracy: 0.5431 - val_loss: 1.1957


INFO:tensorflow:Assets written to: C:\Users\CHAKKA~1\AppData\Local\Temp\tmpz9d2sc_7\assets


INFO:tensorflow:Assets written to: C:\Users\CHAKKA~1\AppData\Local\Temp\tmpz9d2sc_7\assets


Saved artifact at 'C:\Users\CHAKKA~1\AppData\Local\Temp\tmpz9d2sc_7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='input_layer_6')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2027295689712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027295690592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027277149808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027277150688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027296955728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027277150160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027295529792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027295530320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027295528736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2027295527328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  202

c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(





{'k': 16, 'c': 0, 'RAM': 33812, 'Flash': 68304, 'MACC': 1080281, 'max_val_acc': 0.546}




k_16_c_1

81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4034 - loss: 1.4450 - val_accuracy: 0.3751 - val_loss: 1.4953



RAM: 56596, Flash: 152848, MACC: 3961049




{'k': 16, 'c': 1, 'RAM': 'Outside the upper bound', 'Flash': 'Outside the upper bound', 'MACC': 'Outside the upper bound', 'max_val_acc': -3}




Resulting architecture: {'k': 8, 'c': 1, 'RAM': 37716, 'Flash': 96792, 'MACC': 1260281, 'max_val_acc': 0.599}

Model saved to: trained_models\resulting_architecture_k_8_c_1.tflite
Elapsed time (search): 0:01:34.886554

Total models evaluated: 7

Search completed! Model saved at: trained_models\resulting_architecture_k_8_c_1.tflite


In [7]:
# Test the model if found
if path_to_tflite_model and Path(path_to_tflite_model).exists():
    # Create test dataset
    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=str(data_dir),
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=input_shape[0:2],
        shuffle=True,
        seed=42,
        validation_split=0.8,
        subset='validation'
    )
    
    print("Testing model...")
    accuracy = test_tflite_model(path_to_tflite_model, test_ds)
    
    model_size = Path(path_to_tflite_model).stat().st_size
    print(f"Model size: {model_size/1024:.2f} KB")
    print(f"Model accuracy: {accuracy:.4f}")
else:
    print("No model to test.")

Found 3670 files belonging to 5 classes.
Using 2936 files for validation.
Testing model...


c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)



TFLite model test accuracy: 0.6080
Model size: 7.48 KB
Model accuracy: 0.6080
